# CityBikes

Send a request to CityBikes for the city of your choice. 

I submitted this URL via postman:

In order to find the below I used the URL via browser to play around with the JSON with browser filtering.

In [1]:
import requests
import pandas as pd
import json

In [2]:
#url = "http://api.citybik.es/v2/networks/rtc-bike-share"
#url = "http://api.citybik.es/v2/networks/divvy" #chicago
url = "http://api.citybik.es/v2/networks/bay-wheels"

response = requests.get(url)
print(response)

<Response [200]>


In [18]:
#Testing out what a JSON response looks like and pulling various data points
response_json = json.loads(response.text)
country = response_json['network']['location']['country']
city = response_json['network']['location']['city']
station  = response_json['network']['stations'][0]
uid  = response_json['network']['stations'][0]['id']
#slots = response_json['network']['slots']
print(f"city:\n {city}, country: {country}, id:{uid} station details in json: \n{station}")
#print(json.dumps(response_json, indent=2))


city:
 San Francisco Bay Area, CA, country: US, id:d0e8f4f1834b7b33a3faf8882f567ab8 station details in json: 
{'empty_slots': 9, 'extra': {'ebikes': 0, 'has_ebikes': True, 'last_updated': 1702499731, 'payment': ['key', 'creditcard'], 'payment-terminal': True, 'rental_uris': {'android': 'https://sfo.lft.to/lastmile_qr_scan', 'ios': 'https://sfo.lft.to/lastmile_qr_scan'}, 'renting': 1, 'returning': 1, 'slots': 15, 'uid': 'fd89514c-f878-4cd5-8113-8e5beead44de'}, 'free_bikes': 6, 'id': 'd0e8f4f1834b7b33a3faf8882f567ab8', 'latitude': 37.849735, 'longitude': -122.270582, 'name': 'Harmon St at Adeline St', 'timestamp': '2023-12-13T20:38:31.125000Z'}


In [20]:
# Assuming 'response_json' is your JSON data
stations = response_json.get("network", {}).get("stations", [])

for station in stations:
    # each individual station in the array
    # station's properties
    #station_id = station.get("id", "N/A") #if no id then it will say not applicable
    station_name = station.get("name", "N/A")
    station_lat = station.get("latitude",0.00)
    station_lon = station.get("longitude",0.00)
    station_id = station.get("id")
    station_num_bike = station.get("free_bikes","N/A")
    station_slots = station.get("empty_slots")
    total_bikes = station_num_bike + station_slots
    
    # Add more attributes as needed

    print(f"Station Name: {station_name}, Lat: {station_lat}, Lon:{station_lon}, number of bikes: {station_num_bike}, empty slots: {station_slots}, tot bikes: {total_bikes}, id:{station_id}")


Station Name: Harmon St at Adeline St, Lat: 37.849735, Lon:-122.270582, number of bikes: 6, empty slots: 9, tot bikes: 15, id:d0e8f4f1834b7b33a3faf8882f567ab8
Station Name: Fountain Alley at S 2nd St, Lat: 37.33618830029063, Lon:-121.88927650451659, number of bikes: 11, empty slots: 0, tot bikes: 11, id:983514094dd808b1604da2dcfc2d09af
Station Name: Oak St at 1st St, Lat: 37.322124625448566, Lon:-121.88109040260315, number of bikes: 14, empty slots: 7, tot bikes: 21, id:da17603652106fda93da4e255a5b0a22
Station Name: Bestor Art Park, Lat: 37.3236779, Lon:-121.8741186, number of bikes: 11, empty slots: 4, tot bikes: 15, id:7a21c92b3b4cd2f7759107b4fdebf869
Station Name: 5th St at Virginia St, Lat: 37.3259984, Lon:-121.87712, number of bikes: 8, empty slots: 19, tot bikes: 27, id:ce34d38fb230a23c1ced12d1e16df294
Station Name: Locust St at Grant St, Lat: 37.3229796, Lon:-121.8879312, number of bikes: 17, empty slots: 2, tot bikes: 19, id:a3b487ad4ac93ab3e9f9654f87ed8c1e
Station Name: Pierce

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [34]:
### The below returns one result

#attempt to return results on if the server returns success
if response.status_code == 200:
    data = response.json()
    #If the key "network" is present, it returns its value; otherwise, it returns an empty dictionary {}.
    location = data.get("network", {}).get("location", {})
    #stations = data.get("network", {}).get("stations", {})   
    city = location.get("city")
    latitude = location.get("latitude")
    longitude = location.get("longitude")
    uid = data.get("network", {}).get("stations", [{}])[0].get("id", None)
    #bikes = stations.get("free_bikes")
    bikes = data.get("network", {}).get("stations", [{}])[0].get("free_bikes", None)
    
    # Print values
    print(f"City: {city} \nLatitude: {latitude} \nLongitude: {longitude} \nBikes:{bikes}\n {uid}")
else:
    print(f"Error: {response.status_code}")


City: San Francisco Bay Area, CA 
Latitude: 37.7141454 
Longitude: -122.25 
Bikes:6
 d0e8f4f1834b7b33a3faf8882f567ab8


In [42]:
# Attempt to return ALL Results
if response.status_code == 200:
    data = response.json()
    
    # Check if "network" and "stations" keys are present in the response
    network_data = data.get("network", {})
    stations_list = network_data.get("stations", [])
    #empty list to plug into the DF
    stations_info = []
    
    # Loop through stations and extract fields
    for station in stations_list:
        location = network_data.get("location", {})
        city = location.get("city")
        uid = station.get("id", None)
        latitude = station.get("latitude")
        #noticed the lat/long was all the same so it was grabbing the network's latlong not the stations
        #longitude = location.get("longitude")
        longitude = station.get("longitude")
        bikes = station.get("free_bikes", None)
        empty_slots = station.get("empty_slots")
        total_bikes = bikes + empty_slots

        # Append information to the list
        stations_info.append({"City": city, "Latitude": latitude, "Longitude": longitude, "Free_Bikes": bikes, "Empty_Slots": empty_slots, "Total_Bikes": total_bikes, "Id": uid})

    # Create a DataFrame from the list of dictionaries
    
        
        # Print values for each station
        #print(f"City: {city} \nLatitude: {latitude} \nLongitude: {longitude} \nBikes: {bikes}\n---")
else:
    print(f"Error: {response.status_code}")

Put your parsed results into a DataFrame.

In [43]:
df_bikes = pd.DataFrame(stations_info)

In [47]:
#Need to differentiate lat/lon of bikes vs POI
#df_bikes.rename(columns={'Latitude': 'Bike_Lat'}, inplace=True)
#df_bikes.rename(columns={'Longitude': 'Bike_Lon'}, inplace=True)
df_bikes.rename(columns={'Id': 'Bike_Id'}, inplace=True)

In [45]:
df_bikes.head()

,City,Bike_Lat,Bike_Lon,Free_Bikes,Empty_Slots,Total_Bikes,Id
0,"San Francisco Bay Area, CA",37.849735,-122.270582,6,9,15,d0e8f4f1834b7b33a3faf8882f567ab8
1,"San Francisco Bay Area, CA",37.336188,-121.889277,11,0,11,983514094dd808b1604da2dcfc2d09af
2,"San Francisco Bay Area, CA",37.322125,-121.881090,14,7,21,da17603652106fda93da4e255a5b0a22
3,"San Francisco Bay Area, CA",37.323678,-121.874119,11,4,15,7a21c92b3b4cd2f7759107b4fdebf869
4,"San Francisco Bay Area, CA",37.325998,-121.877120,8,19,27,ce34d38fb230a23c1ced12d1e16df294


In [16]:
#save dataframe as csv export to save on API calls
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

In [54]:
#export the dataframe to be accessed in a different notebook
df_bikes.to_csv('..\data\df_bikes.csv', index=False)

In [50]:
df_bikes.columns

Index(['City', 'Bike_Lat', 'Bike_Lon', 'Free_Bikes', 'Empty_Slots',
       'Total_Bikes', 'Bike_Id'],
      dtype='object')

In [51]:

# new col order
column_order = ['Bike_Id', 'City', 'Bike_Lat','Bike_Lon', 'Free_Bikes', 'Empty_Slots','Total_Bikes']

# Reorder columns
df_bikes = df_bikes[column_order]

# modified DataFrame
df_bikes.head()

,Bike_Id,City,Bike_Lat,Bike_Lon,Free_Bikes,Empty_Slots,Total_Bikes
0,d0e8f4f1834b7b33a3faf8882f567ab8,"San Francisco Bay Area, CA",37.849735,-122.270582,6,9,15
1,983514094dd808b1604da2dcfc2d09af,"San Francisco Bay Area, CA",37.336188,-121.889277,11,0,11
2,da17603652106fda93da4e255a5b0a22,"San Francisco Bay Area, CA",37.322125,-121.881090,14,7,21
3,7a21c92b3b4cd2f7759107b4fdebf869,"San Francisco Bay Area, CA",37.323678,-121.874119,11,4,15
4,ce34d38fb230a23c1ced12d1e16df294,"San Francisco Bay Area, CA",37.325998,-121.877120,8,19,27


In [53]:
#32 character unique bikestation id
print(df_bikes['Bike_Id'].str.len())

0      32
1      32
2      32
3      32
4      32
       ..
549    32
550    32
551    32
552    32
553    32
Name: Bike_Id, Length: 554, dtype: int64
